# Reader - Experimento

Este componente utiliza um modelo de QA pré-treinado em Português com o dataset SQuAD v1.1, é um modelo de domínio público disponível em [Hugging Face](https://huggingface.co/pierreguillou/bert-large-cased-squad-v1.1-portuguese).<br>

Seu objetivo é encontrar a resposta de uma ou mais perguntas de acordo com uma lista de contextos distintos.

A tabela de dados de entrada deve possuir uma coluna de contextos, em que cada linha representa um contexto diferente, e uma coluna de perguntas em que cada linha representa uma pergunta a ser realizada. Note que para cada pergunta serão utilizados todos os contextos fornecidos para realização da inferência, e portanto, podem haver bem mais contextos do que perguntas.

Obs: Este componente utiliza recursos da internet, portanto é importante estar conectado à rede para que este componente funcione corretamente.

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [42]:
dataset = "/tmp/data/simple_q&a-12.csv" #@param {type:"string"}
context_column_name = "text" #@param {type:"string", label:"Coluna dos contextos", description:"Esta coluna será utilizada para ler os contextos e aplicar o Reader."}
question_column_name = "question" #@param {type:"string", label:"Coluna das perguntas", description:"Esta coluna será utilizada para identificar as perguntas a serem aplicadas aos contextos."}
answer_column_name = "answer" #@param {type:"string", label:"Coluna das respostas", description:"Esta coluna será utilizada para gerar a resposta produzida por cada pergunta."}
proba_column_name = "answer_score" #@param {type:"string", label:"Coluna da pontuação das respostas", description:"Esta coluna será utilizada para gerar a pontuação de cada resposta produzida."}
expected_column_name = "" #@param {type:"string", label:"Coluna de avaliação", description:"Esta coluna será utilizada para avaliar as respostas produzidas por cada pergunta, espera-se que nesta coluna estejam as respostas esperadas de cada pergunta em sua respectiva linha na tabela. É uma coluna opcional."}
device = "cpu" #@param ["cuda","cpu"] {type:"string",multiple:false,label:"Dispositivo",description:"Tipo de dispositivo para utilizar o componente."}
keep_best = "sim" #@param ["sim","não"] {type:"string",multiple:false,label:"Apenas Melhor Resposta",description:"Manterá apenas as linhas correspondentes à melhor resposta gerada pelo par pergunta e contexto."}

## Leitura do conjunto de dados

O exemplo abaixo faz a leitura de dados tabulares (ex: .csv).<br>
Modifique o código de acordo com o tipo de dado que desejar ler.

In [43]:
import pandas as pd

df = pd.read_csv(dataset)
df

,index,text,question,expected_answer,expected_retriever_answer
0,1,A bola é um objeto utilizado para lazer de uma...,Qual o formato da bola?,esférica,1
1,2,Golfe é um esporte no qual os jogadores usam d...,Qual o formato da bola?,esférica,1
2,3,"Jogo por tacadas[1] (stroke play[2]), também c...",Qual o formato da bola?,esférica,1
3,1,A bola é um objeto utilizado para lazer de uma...,Aonde se pratica golfe?,campo de golfe,2
4,2,Golfe é um esporte no qual os jogadores usam d...,Aonde se pratica golfe?,campo de golfe,2
5,3,"Jogo por tacadas[1] (stroke play[2]), também c...",Aonde se pratica golfe?,campo de golfe,2


## Conteúdo da tarefa

In [44]:
# Assert parameters

assert context_column_name in df.columns, f"Coluna '{context_column_name}' deve ser uma coluna do dataset."
assert question_column_name in df.columns, f"Coluna '{question_column_name}' deve ser uma coluna do dataset."
assert answer_column_name not in df.columns and answer_column_name != "", f"Coluna '{answer_column_name}' não pode ser vazia e nem uma coluna do dataset."
assert proba_column_name not in df.columns and proba_column_name != "", f"Coluna '{proba_column_name}' não pode ser vazia e nem uma coluna do dataset."
assert expected_column_name == "" or expected_column_name in df.columns, f"Coluna '{expected_column_name}' deve ser uma coluna do dataset ou deixar ou campo em branco."
assert device in ['cuda', 'cpu'], f"Dispositivo '{device}' deve ser ou 'cuda' ou 'cpu'."
assert keep_best in ['sim', 'não'], f"Apenas Melhor Resposta deve ser ou 'sim' ou 'não'."

In [45]:
import torch
import math

# Model parameters
model_parameters = {
    'device':torch.device(device) if torch.cuda.is_available() else torch.device('cpu'),
}
    
# Inference parameters
inference_parameters = {
    'context_column_name':context_column_name,
    'question_column_name':question_column_name,
    'expected_column_name':expected_column_name,
    'answer_column_name':answer_column_name,
    'proba_column_name':proba_column_name,
    'input_columns': list(df.columns),
    'output_columns': list(df.columns) + [answer_column_name, proba_column_name],
    'keep_best': keep_best
}

In [5]:
from reader import Reader

# Initilizate Reader
reader = Reader(**model_parameters)

Downloading:   0%|          | 0.00/918 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/506 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [46]:
# Predict answers #

# Iterate over dataset
for idx, row in df.iterrows():
    
    # Get question
    question = row[inference_parameters['question_column_name']]
    
    # Get context
    context = row[inference_parameters['context_column_name']]
        
    # Make prediction
    answer, probability, _ = reader([question], [context])
    
    # Save to df
    df.at[idx, inference_parameters['answer_column_name']] = answer[0]
    df.at[idx, inference_parameters['proba_column_name']] = probability[0]
    
# Retrieve Only Best Answer #

# Initializate best df
best_df = pd.DataFrame(columns=df.columns)

# Get unique questions
unique_questions = df[inference_parameters['question_column_name']].unique()

# Iterate over each unique question
for question in unique_questions:

    # Filter df
    question_df = df[df[inference_parameters['question_column_name']] == question]

    # Sort by score (descending)
    question_df = question_df.sort_values(by=inference_parameters['proba_column_name'], ascending=False).reset_index(drop=True)

    # Append best ansewer to output df
    best_df = pd.concat((best_df,pd.DataFrame(question_df.loc[0]).T)).reset_index(drop=True)

del reader

In [47]:
# Save dataset
if inference_parameters['keep_best'] == 'sim':
    best_df.to_csv(dataset, index=False)
else:
    df.to_csv(dataset, index=False)

In [48]:
from platiagro.metrics_nlp import wrapper
from platiagro.plotting import plot_data_table
# Evaluate translation (Optional)

# Iff is not empty
if inference_parameters['expected_column_name'] != "":
    
    # Initializate calculator
    calculator = wrapper.MetricsCalculator(metrics=['bleu', 'rouge'])
    
    # Get expected/predicted answers
    y_true = list(best_df[inference_parameters['expected_column_name']].values)
    y_pred = list(best_df[inference_parameters['answer_column_name']].values)
    
    # Perform calculations
    scores = calculator.calculate_from_texts(
        y_pred, 
        y_true
    )
    
    # Plot metrics
    plot_data_table(pd.DataFrame(scores, index = ['Valor']))

## Salva resultados da tarefa

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.<br>
Use essa pasta para salvar modelos, metadados e outros resultados.

In [49]:
from joblib import dump

artifacts = {
    "model_parameters": model_parameters,
    "inference_parameters": inference_parameters
}

dump(artifacts, "/tmp/data/reader.joblib")

['/tmp/data/reader.joblib']